# Getting started

### Imports

In [1]:
import pandas as pd
import numpy as np
import random

from akiFlagger import AKIFlagger, generate_toy_data

### Let's start off by creating some toy data. 

The flagger comes with a built-in generator of a toy data set to demonstrate how it works. Simply call the *generate_toy_data()* function

In [13]:
toy = generate_toy_data(num_patients=10**4)
print(toy.shape)
toy.head()

Successfully generated toy data!
(44026, 6)


,mrn,enc,inpatient,admission,time,creat
0,12732,90532,False,2020-06-17 23:40:52,2020-06-14 23:40:52,0.979986
1,12732,83072,False,2020-06-17 23:40:52,2020-06-16 17:40:52,0.869034
2,12732,85627,False,2020-06-17 23:40:52,2020-06-17 05:40:52,1.487043
3,12732,89502,False,2020-06-17 23:40:52,2020-06-17 05:40:52,1.278596
4,12732,90032,False,2020-06-17 23:40:52,2020-06-17 23:40:52,1.105281


In [14]:
%%time
flagger = AKIFlagger(rolling_window = True, creatinine = 'creat')
rw = flagger.returnAKIpatients(toy)
rw.head()

Wall time: 1min 26s


,,mrn,inpatient,admission,creat,rw
enc,time,,,,,
10001,2020-01-25 01:11:24,16802,True,2020-01-21 01:11:24,0.100000,True
10002,2020-05-08 00:22:09,18266,True,2020-05-05 18:22:09,1.778543,False
10004,2020-03-07 13:08:14,16228,False,2020-03-09 19:08:14,1.163156,True
10005,2020-02-25 19:44:48,12027,True,2020-02-25 13:44:48,1.173645,False
10010,2020-03-14 21:04:03,10030,False,2020-03-17 09:04:03,0.771018,True


In [15]:
%%time
flagger = AKIFlagger(back_calculate = True, creatinine = 'creat')
bc = flagger.returnAKIpatients(toy)
bc.head()

Wall time: 1min 17s


,,mrn,inpatient,admission,creat,bc
enc,time,,,,,
90532,2020-06-14 23:40:52,12732,False,2020-06-17 23:40:52,0.979986,False
83072,2020-06-16 17:40:52,12732,False,2020-06-17 23:40:52,0.869034,False
85627,2020-06-17 05:40:52,12732,False,2020-06-17 23:40:52,1.487043,False
89502,2020-06-17 05:40:52,12732,False,2020-06-17 23:40:52,1.278596,False
90032,2020-06-17 23:40:52,12732,False,2020-06-17 23:40:52,1.105281,False
...,...,...,...,...,...,...
49132,2020-03-30 16:26:24,10981,True,2020-03-26 10:26:24,0.924554,False
10690,2020-01-28 14:50:47,10508,False,2020-02-01 20:50:47,1.612890,False
57919,2020-01-29 08:50:47,10508,False,2020-02-01 20:50:47,1.508203,False
